In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import (
    RBF, Matern, RationalQuadratic, ExpSineSquared, WhiteKernel, 
    ConstantKernel as C
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score


class GaussianProcessSolver:
    """
    A class for Gaussian Process regression that allows for customization
    of kernels and optimization of hyperparameters.
    """
    
    def __init__(self, kernel_type='rbf', optimize_hyperparams=True, random_state=None):
        """
        Initialize the Gaussian Process Solver.
        
        Parameters:
        -----------
        kernel_type : str, default='rbf'
            The type of kernel to use. Options: 'rbf', 'matern', 'rational_quadratic', 
            'exp_sine_squared', or 'composite'.
        optimize_hyperparams : bool, default=True
            Whether to optimize the hyperparameters during fitting.
        random_state : int or None, default=None
            Random seed for reproducibility.
        """
        self.kernel_type = kernel_type
        self.optimize_hyperparams = optimize_hyperparams
        self.random_state = random_state
        self.model = None
        self._set_kernel()
        
    def _set_kernel(self):
        """Set the kernel based on the specified kernel_type."""
        if self.kernel_type == 'rbf':
            # Radial Basis Function kernel
            self.kernel = C(1.0) * RBF(length_scale=1.0) + WhiteKernel(noise_level=0.1)
        elif self.kernel_type == 'matern':
            # Matérn kernel
            self.kernel = C(1.0) * Matern(length_scale=1.0, nu=1.5) + WhiteKernel(noise_level=0.1)
        elif self.kernel_type == 'rational_quadratic':
            # Rational Quadratic kernel
            self.kernel = C(1.0) * RationalQuadratic(length_scale=1.0, alpha=0.1) + WhiteKernel(noise_level=0.1)
        elif self.kernel_type == 'exp_sine_squared':
            # Exponential Sine Squared kernel (periodic)
            self.kernel = C(1.0) * ExpSineSquared(length_scale=1.0, periodicity=1.0) + WhiteKernel(noise_level=0.1)
        elif self.kernel_type == 'composite':
            # A composite kernel combining RBF and Matérn
            self.kernel = C(1.0) * (
                RBF(length_scale=1.0) + 
                Matern(length_scale=1.0, nu=1.5)
            ) + WhiteKernel(noise_level=0.1)
        else:
            raise ValueError(f"Unsupported kernel type: {self.kernel_type}")
            
        # Initialize the Gaussian Process Regressor with the chosen kernel
        self.model = GaussianProcessRegressor(
            kernel=self.kernel,
            alpha=1e-10,  # numerical stability
            normalize_y=True,
            n_restarts_optimizer=10,
            random_state=self.random_state,
            optimizer=None if not self.optimize_hyperparams else 'fmin_l_bfgs_b'
        )
    
    def fit(self, X, y):
        """
        Fit the Gaussian Process model to the training data.
        
        Parameters:
        -----------
        X : array-like of shape (n_samples, n_features)
            Training input samples.
        y : array-like of shape (n_samples,)
            Target values.
            
        Returns:
        --------
        self : object
            Returns self.
        """
        if X.ndim == 1:
            X = X.reshape(-1, 1)
            
        self.model.fit(X, y)
        
        # Store the optimized kernel parameters
        self.optimized_kernel = self.model.kernel_
        
        return self
    
    def predict(self, X_test, return_std=False, return_cov=False):
        """
        Make predictions using the fitted Gaussian Process model.
        
        Parameters:
        -----------
        X_test : array-like of shape (n_samples, n_features)
            Test input samples.
        return_std : bool, default=False
            If True, return the standard deviation of the prediction along with the mean.
        return_cov : bool, default=False
            If True, return the covariance of the prediction along with the mean.
            
        Returns:
        --------
        y_pred : array-like of shape (n_samples,)
            Predicted mean values.
        y_std : array-like of shape (n_samples,), optional
            Standard deviation of predictive distribution at test points.
        y_cov : array-like of shape (n_samples, n_samples), optional
            Covariance of joint predictive distribution at test points.
        """
        if X_test.ndim == 1:
            X_test = X_test.reshape(-1, 1)
            
        if return_std and return_cov:
            raise ValueError("Cannot return both std and cov.")
        
        if return_std:
            return self.model.predict(X_test, return_std=True)
        elif return_cov:
            return self.model.predict(X_test, return_cov=True)
        else:
            return self.model.predict(X_test)
    
    def score(self, X, y):
        """
        Return the coefficient of determination R^2 of the prediction.
        
        Parameters:
        -----------
        X : array-like of shape (n_samples, n_features)
            Test input samples.
        y : array-like of shape (n_samples,)
            True target values.
            
        Returns:
        --------
        score : float
            R^2 of the prediction.
        """
        if X.ndim == 1:
            X = X.reshape(-1, 1)
            
        return self.model.score(X, y)
    
    def get_kernel_params(self):
        """
        Get the optimized kernel parameters.
        
        Returns:
        --------
        params : dict
            Dictionary containing the optimized kernel parameters.
        """
        if hasattr(self, 'optimized_kernel'):
            return self.optimized_kernel.get_params()
        else:
            return self.kernel.get_params()
    
    def plot_fit(self, X_train, y_train, X_test=None, y_test=None, n_samples=100, 
                 plot_samples=False, figsize=(10, 6)):
        """
        Plot the Gaussian Process model fit.
        
        Parameters:
        -----------
        X_train : array-like of shape (n_samples, n_features)
            Training input samples.
        y_train : array-like of shape (n_samples,)
            Training target values.
        X_test : array-like of shape (n_test_samples, n_features), optional
            Test input samples. If not provided, will use a linspace over the range of X_train.
        y_test : array-like of shape (n_test_samples,), optional
            Test target values. Only used for plotting if X_test is provided.
        n_samples : int, default=100
            Number of sample points to use for prediction if X_test is not provided.
        plot_samples : bool, default=False
            Whether to plot sample functions from the posterior.
        figsize : tuple, default=(10, 6)
            Figure size.
            
        Returns:
        --------
        fig : matplotlib.figure.Figure
            The figure containing the plot.
        """
        if X_train.ndim == 1:
            X_train = X_train.reshape(-1, 1)
        
        # Create prediction points if X_test is not provided
        if X_test is None:
            X_min, X_max = X_train.min(), X_train.max()
            X_test = np.linspace(X_min - 0.1 * (X_max - X_min), 
                                 X_max + 0.1 * (X_max - X_min), 
                                 n_samples).reshape(-1, 1)
        elif X_test.ndim == 1:
            X_test = X_test.reshape(-1, 1)
        
        # Make predictions
        y_mean, y_std = self.model.predict(X_test, return_std=True)
        
        # Create the figure
        fig, ax = plt.subplots(figsize=figsize)
        
        # Plot training data
        ax.scatter(X_train, y_train, color='blue', label='Training data')
        
        # Plot test data if provided
        if y_test is not None:
            ax.scatter(X_test, y_test, color='green', label='Test data')
        
        # Sort X_test for proper plotting
        sort_idx = np.argsort(X_test.flatten())
        X_test_sorted = X_test[sort_idx]
        y_mean_sorted = y_mean[sort_idx]
        
        # Plot the mean prediction
        ax.plot(X_test_sorted, y_mean_sorted, color='red', label='Mean prediction')
        
        if plot_samples:
            # Plot sample functions from the posterior
            samples = self.model.sample_y(X_test_sorted, n_samples=10)
            for i in range(samples.shape[1]):
                ax.plot(X_test_sorted, samples[:, i], color='gray', alpha=0.3, 
                       linewidth=0.8)
        else:
            # Plot the uncertainty intervals
            y_std_sorted = y_std[sort_idx]
            ax.fill_between(X_test_sorted.flatten(), 
                           y_mean_sorted - 2 * y_std_sorted,
                           y_mean_sorted + 2 * y_std_sorted, 
                           alpha=0.2, color='red', label='±2 std')
        
        # Set labels and title
        ax.set_xlabel('Time')
        ax.set_ylabel('Viscosity')
        ax.set_title(f'Gaussian Process Regression with {self.kernel_type.capitalize()} Kernel')
        ax.legend()
        
        return fig
    
    def evaluate(self, X_test, y_test):
        """
        Evaluate the model performance on test data.
        
        Parameters:
        -----------
        X_test : array-like of shape (n_samples, n_features)
            Test input samples.
        y_test : array-like of shape (n_samples,)
            True target values.
            
        Returns:
        --------
        metrics : dict
            Dictionary containing evaluation metrics.
        """
        if X_test.ndim == 1:
            X_test = X_test.reshape(-1, 1)
            
        y_pred = self.predict(X_test)
        
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        r2 = r2_score(y_test, y_pred)
        
        return {
            'mse': mse,
            'rmse': rmse,
            'r2': r2
        }
    
    def sample_posterior(self, X, n_samples=20):
        """
        Sample functions from the posterior distribution.
        
        Parameters:
        -----------
        X : array-like of shape (n_samples, n_features)
            Input points.
        n_samples : int, default=10
            Number of samples to draw.
            
        Returns:
        --------
        samples : array-like of shape (n_samples, n_input_points)
            Sampled function values at the input points.
        """
        if X.ndim == 1:
            X = X.reshape(-1, 1)
            
        return self.model.sample_y(X, n_samples=n_samples)